<a href="https://colab.research.google.com/github/lsjlsj5846/error-unifiedQA/blob/main/FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CommonsenseQA의 오답 유형 분석: Unified-QA 모델을 중심으로
인문대학 언어학과 | 2018-18450 이소진

In [2]:
!pip install transformers
!pip install ratsnlp
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 56.4 MB/s 
     |████████████████████████████████| 182 kB 84.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 2.8 MB 8.7 MB/s 
     |████████████████████████████████| 582 kB 65.5 MB/s 
     |████████████████████████████████| 57 kB 6.8 MB/s 
     |████████████████████████████████| 512 kB 72.7 MB/s 
     |████████████████████████████████| 880 kB 61.1 MB/s 
     |████████████████████████████████| 3.3 MB 53.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4d1b7d24127fc97b3f5881b8c927820bd51142478f1fa2a3b4e5c00a2b50ed0f
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659b

In [3]:
!git clone https://github.com/lsjlsj5846/unifiedqa.git

Cloning into 'unifiedqa'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 353 (delta 57), reused 37 (delta 37), pack-reused 280
Receiving objects: 100% (353/353), 28.88 MiB | 45.77 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")
  print("Running on the CPU")

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-v2-t5-large-1251000" # you can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

In [11]:
!gdown https://drive.google.com/uc?id=1Dgl3wwCQXYN5feRVSrt83icF7KBwVFvN
!mkdir data
!mv /content/dev_rand_split.jsonl /content/data

Downloading...
From: https://drive.google.com/uc?id=1Dgl3wwCQXYN5feRVSrt83icF7KBwVFvN
To: /content/dev_rand_split.jsonl
100% 472k/472k [00:00<00:00, 69.4MB/s]
mkdir: cannot create directory ‘data’: File exists


In [12]:
import random
import jsonlines as js

json_file_path = "/content/data/dev_rand_split.jsonl"

answer_list = []
question_list = []

with js.open(json_file_path) as f:
  for line in f:
    answer_list.append(line['answerKey'].lower())
    question_list.append(line['question'])

In [13]:
sample_questions = []
sample_choices = []
sample_answers = []

for i in range(100):
  index = random.randint(0, 1220) # Number of total questions in dataset is 1221
  
  sample_answers.append(answer_list[index])
  
  result = []

  result.append(question_list[index]['stem'])
  result.append('\n')
  
  choice = question_list[index]['choices']
  sample_choices.append(choice)
  for j in range(5):
    result.append('(')
    result.append(choice[j]['label'].lower())
    result.append(')')
    result.append(' ')
    result.append(choice[j]['text'])
    result.append(' ')
  
  result_str = ''.join(result)
  result_str = result_str.strip()

  sample_questions.append(result_str)

In [14]:
total = 0

for i, question in enumerate(sample_questions):
  total += 1
  result = run_model(question)

  predicted = ''

  if sample_choices[i][0]['text'] == result[0]:
    predicted = 'a'
  elif sample_choices[i][1]['text'] == result[0]:
    predicted = 'b'
  elif sample_choices[i][2]['text'] == result[0]:
    predicted = 'c'
  elif sample_choices[i][3]['text'] == result[0]:
    predicted = 'd'
  elif sample_choices[i][4]['text'] == result[0]:
    predicted = 'e'
  else:
    predicted = 'UNKNOWN'
  
  if predicted != sample_answers[i]:
    print("Q" + str(total))
    print(question)
    print("result: " + str(result[0]))
    print("answer: " + str(sample_answers[i]))
    print()

Q1
Athletes soak in hot tubs to relieve what after playing baseball?
(a) fame (b) errors (c) pain (d) strikes (e) sore muscles
result: pain
answer: e

Q5
What do people need to do to change their lives?
(a) face problems (b) better themselves (c) pay bills (d) become disillusioned (e) eat chicken
result: better themselves
answer: a

Q9
What do you go to see for live entertainment?
(a) movie (b) show (c) concert venue (d) casino (e) theatre
result: concert venue
answer: b

Q11
A supermarket is uncommon in what type of collection of shops?
(a) strip mall (b) city or town (c) shoppingcentre (d) boutique (e) vermont
result: boutique
answer: a

Q14
how can i store cooked steak?
(a) oven (b) freezer (c) plate (d) tupperware (e) grill
result: tupperware
answer: b

Q17
If chewing food is difficult for you, what is a possible reason?
(a) broken jaw (b) sore mouth (c) eating (d) good digestion (e) avoiding choking
result: broken jaw
answer: b

Q18
Where is there a telephone book in almost every 